In [1]:
import os
import pickle

import torch as T
import yaml
from tqdm import tqdm
from icecream import ic

import sys

sys.path.append('/Users/marcelloceresini/github/FSOD_CenterNet/src')
from data_pipeline import DatasetsGenerator
from model import Model
from evaluation import Evaluate


def load_settings(settings_path: str):
    with open(settings_path, 'r') as f:
        return yaml.safe_load(f)
    
settings = '../settings/model_testing_debug_check.yaml'
    
config = load_settings(settings)

os.makedirs(config['training']['save_training_info_dir'], exist_ok=True)

debug_mode = config['debug']['debug_mode_active']
device = config['device']

# Dataset generator. Only one of these has to be instantiated. It always returns
dataset_gen = DatasetsGenerator(config)

if config['evaluation']['base']:

    # Instantiate the model (only the base part)
    model = Model(config, n_base_classes=len(dataset_gen.train_base.cats))

    model = model.to(device)
    model.load_state_dict(T.load('../../data/weights/from_server/best_model_fix_stoic.pt', map_location="cpu"))

    # Use the dataset generator to generate the base set
    dataset_base_train, dataset_base_val, dataset_base_test = dataset_gen.get_base_sets_dataloaders(
        config['training']['batch_size'], config['training']['num_workers'],
        config['training']['pin_memory'], config['training']['drop_last'], 
        shuffle=True
    )

    print("Before Evaluate")

    metrics_test = Evaluate(
                model, 
                dataset_base_test, 
                prefix="test/",
                device=device,
                config=config
            )(is_novel=False)
    
    print("After Evaluate")

    with open(os.path.join(config['training']['save_training_info_dir'], 
                           config['training']['base_stats_save_name']), 'wb') as f:
        pickle.dump(metrics_test, f)

else:
    raise NotImplementedError("Novel evaluation not implemented yet")


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=2.05s)
creating index...
index created!
Skipping instantating novel head


RuntimeError: Error(s) in loading state_dict for Model:
	Missing key(s) in state_dict: "head_regressor.conv3.weight", "head_regressor.conv3.bias", "head_regressor.conv4.weight", "head_regressor.conv4.bias", "head_base_heatmap.conv2.weight", "head_base_heatmap.conv2.bias", "head_base_heatmap.conv3.weight", "head_base_heatmap.conv3.bias". 
	size mismatch for head_regressor.conv2.weight: copying a param with shape torch.Size([4, 64, 1, 1]) from checkpoint, the shape in current model is torch.Size([64, 64, 3, 3]).
	size mismatch for head_regressor.conv2.bias: copying a param with shape torch.Size([4]) from checkpoint, the shape in current model is torch.Size([64]).

In [ ]:
ic(metrics_test)